In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import scipy.sparse.linalg as spla
from pytenet.krylov_time_evo import ED_time_evo, Krylov_evo_using_vecs_single_step, Krylov_time_evo_using_vecs

In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [5]:
#ref Hamiltonian as matrix
H_ref = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_thc = scipy.io.mmread('../data_water/H_water_thc.mtx').tocsr()

#initial state
filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
with open(filename, 'rb') as file:
    initial_state = pickle.load(file)

#thc mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)
    

In [4]:
psi_krylov_ref = Krylov_time_evo_using_vecs(H_ref, 6, initial_state.as_vector(), 5, 0.05)
psi_ed = ED_time_evo(H_ref, initial_state.as_vector(), 0.05)
print(norm(psi_krylov_ref - psi_ed))

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov_time_evo.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  TN[i, j] = np.vdot(Krylov_vector_list[i], H@Krylov_vector_list[j])


1.0141732533164143e-10


In [ ]:
'''
for i in range(n):
first update foldername
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)
then calculate time-evolved state (with re-ortho)
'''
#不需要创造很多文件夹储存文件，对每个文件好好命名然后都存在同一个文件夹即可